# TMvis
# Database for Transmembrane Protein Annotation and Visualization
## Table of contents
* [Step 1. Install dependencies](#first-bullet)
* [Step 2. Provide path to TMvis dataset](#second-bullet)
* [Step 3. Explore TMvis in 3D](#third-bullet)
    * [TMbed](#tmbed)
    * [ANVIL](#anvil)
    * [PPM3](#ppm)

## 1. Install dependencies <a class="anchor" id="first-bullet"></a>
Press `Run` button on the top of the Notebook to install the neccessary dependencies. Make sure that the cell did not output any errors (warnings are okay) before you continue.

In [ ]:
import sys
# 3D visualization of protein molecules
!{sys.executable} -m pip install py3Dmol
# iPython widgets e.g. sliders
!{sys.executable} -m pip install ipywidgets
# fasta files handling
!{sys.executable} -m pip install Bio

## 2. Provide path to a folder with TMvis dataset<a class="anchor" id="second-bullet"></a>
Run the following cell to connect the Notebook to TMvis dataset. The dataset folder should have the following structure: 
```
db                           
|                                    
├── alpha                                 
│   │                                 
│   ├── pLDDT90F1
|   |   |
|   |   └──A0A087X1C5                                 
|   │       ├── A0A087X1C5.fasta                                 
|   │       ├── AF-A0A087X1C5-F1-model_v2.pdb                                 
|   │       ├── AF-A0A087X1C5-F1-model_v2.cif                                 
|   │       ├── AF-A0A087X1C5-F1-model_v2_ANVIL.pdb                                 
|   │       └── AF-A0A087X1C5-F1-model_v2_ppm.PDB                                 
|   |       └── ...                                    
└── beta                                 
    └── pLDDT90F1
        └── P45880 
        └── ...
```
**Please make sure to provide a correct path to the directory where TMvis dataset was downloaded.**        

In [ ]:
import glob
import os

# [TODO] Provide path to folder with transmembrane proteins called db (without including the db folder itself)
home_path = "./../results/current/"
file_path = os.path.join(home_path, 'db/')

# all Uniprot IDs of TMvis
alpha = sorted(os.listdir(os.path.join(file_path, 'alpha/pLDDT90F1')))
beta = sorted(os.listdir(os.path.join(file_path, 'beta/pLDDT90F1')))
print("# proteins in total: ", len(alpha + beta))

## 3. Explore TMvis in 3D<a class="anchor" id="third-bullet"></a>
Run the following cells to visualize transmembrane proteins (TMPs) embedded in a membrane. To select a protein - start typing in a Uniprot ID to a dedicated field under the cell, then available Uniprot IDs from the TMvis dataset will appear.

## Transmembrane Protein Prediction with Embeddings<a class="anchor" id="tmbed"></a>
## TMbed 
A novel method predicting alpha helical and beta barrel TMPs, their transmembrane segments and overall inside/outside topology relative to the membrane. Utilizing embeddings generated by language models from natural language processing and trained on protein sequences, it can predict whole proteomes within an hour while maintaining a prediction accuracy equal to or better than methods using evolutionary information (sequence profiles or alignments)[3]. Furthermore, in this section you can compare TMbed predictions to ANVIL[1] or PPM3[2] membrane assignment.          
                         
**TMbed color code**     
<font color='green'>IN --> OUT orientation</font>       
<font color='blue'>OUT --> IN orientation</font>     
                
**ANVIL and PPM color code**      
<font color='red'>membrane</font>  

In [ ]:
from Bio import SeqIO
import py3Dmol
import glob
import ipywidgets as widgets

def show_tmbed(uniprotID, overlay_with=["ppm3", "anvil"], membr_opacity=0.7):
    if overlay_with == 'anvil':
        if uniprotID in alpha:
                pdb_file = f'{file_path}/alpha/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ANVIL.pdb'
                fasta_file = f'{file_path}/alpha/pLDDT90F1/{uniprotID}/{uniprotID}.fasta'
        else:
                pdb_file = f'{file_path}/beta/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ANVIL.pdb'
                fasta_file = f'{file_path}/beta/pLDDT90F1/{uniprotID}/{uniprotID}.fasta'
    else:
        # ppm3
        if uniprotID in alpha:
                pdb_file = f'{file_path}/alpha/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ppm.PDB'
                fasta_file = f'{file_path}/alpha/pLDDT90F1/{uniprotID}/{uniprotID}.fasta'
        else:
                pdb_file = f'{file_path}/beta/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ppm.PDB'
                fasta_file = f'{file_path}/beta/pLDDT90F1/{uniprotID}/{uniprotID}.fasta'

    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

    seq_obj = list(SeqIO.parse(fasta_file, "fasta"))[0]

  # FASTA file contains ID, protein sequence and transmembrane residue prediction (3 lines)
  # len(protein sequence) = len(transmembrane prediction) = num(residues)
  # SeqIO parses everything from the second line after > as a sequence -> aa sequence and
  # transmembrane prediction are captured together and needed to be splited from each other
    seq_len = len(seq_obj.seq)
    middle = seq_len // 2
    tm_bed_seq = seq_obj.seq[middle:seq_len]

    if overlay_with == 'anvil':
        with open(pdb_file) as ifile:
            system = "".join([x for x in ifile])
            view.addModelsAsFrames(system)
            num_atom = 0
            for line in system.split("\n"):
                split = line.split()
                # skip comments in the PDB file
                if (len(split) == 0) or (split[0] not in ["ATOM", "HETATM90002", "HETATM90001"]):
                      continue
                else: 
                    # membrane atoms definition
                    if split[0] == "HETATM90002" or split[0] == "HETATM90001":
                        view.setStyle({'model': -1, 'resn': 'DUM'}, {"sphere": {'color': 'red', 'opacity': membr_opacity}})
                    else:
                        # protein definition
                        # extract the atoms residue
                        res_num = split[5]
                        res_type = tm_bed_seq [int(res_num) - 1]
                        if res_type == '.':
                              atom_color = 'grey'
                        elif res_type in ['B', 'H']:
                              atom_color = 'green'
                        elif res_type in ['b', 'h']:
                              atom_color = 'blue'
                        else:
                              atom_color = 'pink'
                    num_atom += 1
                    view.setStyle({'model': -1, 'serial': num_atom}, {"cartoon": {'color': atom_color}})
    else:
          with open(pdb_file) as ifile:
            system = "".join([x for x in ifile])
            view.addModelsAsFrames(system)
            num_atom = 0
            for line in system.split("\n"):
                split = line.split()
                # skip comments
                if (len(split) == 0) or (split[0] not in ["ATOM", "HETATM"]):
                    continue
                else: 
                    if split[0] == "HETATM":
                        view.setStyle({'model': -1, 'resn': 'DUM'}, {"sphere": {'color': 'red', 'opacity': membr_opacity}})
                    else:
                        # protein definition
                        # extract the atoms residue
                        res_num = split[5]
                        res_type = tm_bed_seq [int(res_num) - 1]
                        #print(res_type, ":", res_num)
                        if res_type == '.':
                            atom_color = 'grey'
                        elif res_type in ['B', 'H']:
                            atom_color = 'green'
                        elif res_type in ['b', 'h']:
                            atom_color = 'blue'
                        else:
                            atom_color = 'pink'
                        num_atom += 1
                        view.setStyle({'model': -1, 'serial': num_atom}, {"cartoon": {'color': atom_color}})
    view.zoomTo()
    assert (len(tm_bed_seq) == int(res_num)), "Number of residues is not equal to the length of the TMbed prediction string"
    return view.show()

widgets.interact(show_tmbed, uniprotID=widgets.Combobox(
    options=alpha+beta,
    value=alpha[0]));

## Assignment aNd VIsualization of the Lipid bilayer<a class="anchor" id="anvil"></a>
## ANVIL
ANVIL[1] is aimed to assign membrane boundaries to a protein 3D structure, by using the spatial coordinates of the alpha carbons. It can also process coarse-grained models of protein structures. The algorithm follows an approach that treats the problem of membrane assignment as a binary classification. The software is implemented in Python and is freely available under a CeCILL License. In output, ANVIL writes a PDB file that contains the coordinates of the membrane (defined as heteroatoms and visualized here in red).           
A protein is colored based on AlphaFold [4] pLDDT score or can be changed to rainbow color scheme. AlphaFold produces a per-residue confidence score (pLDDT) between 0 and 100. Some regions below 50 pLDDT may be unstructured in isolation.        

<font color='blue'> Very high (pLDDT > 90)</font>      
<font color='aqua'>  Confident (90 > pLDDT > 70)</font>     
<font color='yellow'>  Low (70 > pLDDT > 50)</font>     
<font color='orange'>  Very low (pLDDT < 50)</font>       

In [ ]:
def show_anvil_pdb(uniprotID, show_sidechains=False, show_mainchains=False, show_heteroatoms=True, show_surface=True, color="lDDT"):
    
    if uniprotID in alpha:
        pdb_file = f'{file_path}/alpha/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ANVIL.pdb'
    else:
        pdb_file = f'{file_path}/beta/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ANVIL.pdb'
            
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    with open(pdb_file) as ifile:
        system = "".join([x for x in ifile])
        view.addModelsAsFrames(system)

    if color == "lDDT":
        view.setStyle({'model': -1}, {'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
        
    elif color == "rainbow":
        view.setStyle({'model': -1}, {'cartoon': {'color':'spectrum'}})
        
    if show_heteroatoms:
        for line in system.split("\n"):
            split = line.split()
            if (len(split) == 0) or (split[0] not in ["ATOM", "HETATM90002", "HETATM90001"]):
                  continue
            if split[0] == "HETATM90002" or split[0] == "HETATM90001":
                color = 'red'
                view.setStyle({'model': -1, 'resn': 'DUM'}, {"sphere": {'color': color, 'opacity':0.7}})

    if show_sidechains:
        BB = ['C','O','N']
        view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                            {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]}, {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                            {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    if show_mainchains:
        BB = ['C','O','N','CA']
        view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

    if show_surface:
        view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'grey'}, {'model': -1, 'resn': 'DUM', 'invert': 'True'})
        
    view.zoomTo()
    return view.show()


widgets.interact(show_anvil_pdb, uniprotID=widgets.Combobox(
    options=alpha+beta,
    value=alpha[0]));

## Positioning of proteins in flat and curved membranes<a class="anchor" id="ppm"></a>
## PPM3.0
*   Positioning of transmembrane and peripheral proteins and peptides in membranes and micelles
*   Calculation of binding modes and membrane affinity of proteins taking into account the influence of hydrophobic matching and curvature stress
* prediction of protein-induced membrane deformations
* calculations in flat artificial and natural membranes
* calculations in curved membranes with adjustable radii of curvature
* calculations in spherical micelles of different sizes
* positioning of proteins spanning two parallel membranes (e.g. outer and inner membranes of Gram-negative bacteria)
* approximation of deformed membrane surfaces by several (up to four) flat surfaces
* using polarity profiles along the membrane normal for the DOPC (1,2-dioleoyl-sn-glycero-3-phosphocholine) bilayerusing hydrophobic thicknesses (± 5Å) of artificial lipid bilayers (experimental values) and of different biological membranes (average values obtained from the large-scale analysis of transmembrane proteins from the OPM database [2]       


A protein is colored based on AlphaFold [4] pLDDT score or can be changed to rainbow color scheme. AlphaFold produces a per-residue confidence score (pLDDT) between 0 and 100. Some regions below 50 pLDDT may be unstructured in isolation.        

<font color='blue'> Very high (pLDDT > 90)</font>      
<font color='aqua'>  Confident (90 > pLDDT > 70)</font>     
<font color='yellow'>  Low (70 > pLDDT > 50)</font>     
<font color='orange'>  Very low (pLDDT < 50)</font>       

In [ ]:
def show_ppm_pdb(uniprotID, show_sidechains=False, show_mainchains=False, show_heteroatoms=True, show_surface=True, color="lDDT"):
    if uniprotID in alpha:
        pdb_file = f'{file_path}/alpha/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ppm.PDB'
    else:
        pdb_file = f'{file_path}/beta/pLDDT90F1/{uniprotID}/AF-{uniprotID}-F1-model_v2_ppm.PDB'
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    
    with open(pdb_file) as ifile:
        system = "".join([x for x in ifile])
    view.addModelsAsFrames(system)
    
    if color == "lDDT":
        view.setStyle({'model': -1}, {'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
    elif color == "rainbow":
        view.setStyle({'model': -1}, {'cartoon': {'color':'spectrum'}})
    
    if show_sidechains:
        BB = ['C','O','N']
        view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                            {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]}, {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                            {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
    if show_mainchains:
        BB = ['C','O','N','CA']
        view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    
    if show_surface:
        view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'grey'}, {'model': -1, 'resn': 'DUM', 'invert': 'True'})
    
    if show_heteroatoms:
        for line in system.split("\n"):
            split = line.split()
            if (len(split) == 0) or (split[0] not in ["ATOM", "HETATM"]):
                  continue
            if split[0] == "HETATM":
                color = 'red'
                view.setStyle({'model': -1, 'resn': 'DUM'}, {"sphere": {'color': color, 'opacity':0.7}})
    view.animate({'loop': 'backAndForth'})
    view.zoomTo()
    return view.show()

widgets.interact(show_ppm_pdb, uniprotID=widgets.Combobox(
    options=alpha+beta,
    value=alpha[0]));


# References
1. Postic, Guillaume, Yassine Ghouzam, Vincent Guiraud, and Jean-Christophe Gelly. 2016. “Membrane Positioning for High- and Low-Resolution Protein Structures through a Binary Classification Approach.” Protein Engineering, Design & Selection: PEDS 29 (3): 87–91.        
2. Lomize, Mikhail A., Irina D. Pogozheva, Hyeon Joo, Henry I. Mosberg, and Andrei L. Lomize. 2012. “OPM Database and PPM Web Server: Resources for Positioning of Proteins in Membranes.” Nucleic Acids Research 40 (Database issue): D370–76.                                 
3. Bernhofer, Michael, and Burkhard Rost. 2022. “TMbed – Transmembrane Proteins Predicted through Language Model Embeddings.” bioRxiv. https://doi.org/10.1101/2022.06.12.495804.                            
4. Tunyasuvunakool, Kathryn, Jonas Adler, Zachary Wu, Tim Green, Michal Zielinski, Augustin Žídek, Alex Bridgland, et al. 2021. “Highly Accurate Protein Structure Prediction for the Human Proteome.” Nature 596 (7873): 590–96.     
5. Varadi, Mihaly, Stephen Anyango, Mandar Deshpande, Sreenath Nair, Cindy Natassia, Galabina Yordanova, David Yuan, et al. 2022. “AlphaFold Protein Structure Database: Massively Expanding the Structural Coverage of Protein-Sequence Space with High-Accuracy Models.” Nucleic Acids Research 50 (D1): D439–44.